# Analysis of flow at Lees Ferry on the Colorado River

Import the necessary Python libraries for the analysis

In [6]:
from suds.client import Client
from pandas import Series
import matplotlib.pyplot as plt
import matplotlib.ticker

Create the inputs needed for the web service call

In [7]:
wsdlURL = 'http://hydroportal.cuahsi.org/nwisuv/cuahsi_1_1.asmx?WSDL'
siteCode = 'NWISUV:09380000'    # gage ID for Colorado River at Lees Ferry
variableCode = 'NWISUV:00060'   # USGS code for discharge
beginDate = '2018-08-01'        # Time window for the plots
endDate = '2018-11-13'

Create a dataframe based off the web service call inputs

In [8]:
NWIS = Client(wsdlURL).service
# Call the GetValuesObject method to return datavalues
response = NWIS.GetValuesObject(siteCode, variableCode, beginDate, endDate)
# Get the site's name from the response
siteName = response.timeSeries[0].sourceInfo.siteName
# Create some blank lists in which to put the values and their dates
a = []  # The values
b = []  # The dates
# Get the values and their dates from the web service response
values = response.timeSeries[0].values[0].value
# Loop through the values and load into the blank lists using append
for v in values:
    a.append(float(v.value))
    b.append(v._dateTime)

# Create a Pandas Series object from the lists
# Set the index of the Series object to the dates
ts = Series(a, index=b)

Resample by the day and find the average, minimum, and maximum flow.

In [9]:
tsAvg = ts.resample(rule='1D', base=0).mean()   # The daily average value for the 15 minute values
tsMin = ts.resample(rule='1D', base=0).min()    # The minimum value for that day
tsMax = ts.resample(rule='1D', base=0).max()    # The maximum value for that day

Create a plot that displays the daily average, minimum, and maximum flows seen at Lees Ferry

In [10]:
# Generate a plot of the data subsets
fig = plt.figure(figsize=(6.5,7)) # figsize(width,height)
ax = fig.add_subplot(1, 1, 1)
# Add data sets to the plot
ts.plot(ax=ax, use_index=True, style='-', label='15-minute flow', color='lightgrey')
tsAvg.plot(ax=ax, kind='line', use_index=True, style='-',
           marker='^', label='Average Daily Flow', color='blue')
tsMax.plot(ax=ax, kind='line', use_index=True, style='-',
           marker='.', label='Maximum Daily Flow', color='red')
tsMin.plot(ax=ax, kind='line', use_index=True, style='-',
           marker='.', label='Minimum Daily Flow', color='black')

# Add commas to the values in the y-axis
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y), ',')))
# Add labels to the plot
ax.set_ylabel('Discharge, cubic feet per second')
ax.set_xlabel('Date')
# Add a grid
ax.grid(True)
# Add a legend
legend = ax.legend(loc='upper left', shadow=True)
# Add a title
plt.title('Flow at %s' % (siteName))
fig.tight_layout()
# Make sure the plot displays
plt.show()